# ELAIS-S1 master catalogue: Flags

In [ ]:
import numpy as np
from astropy.table import Table

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [ ]:
SUFFIX = "20180416"
FIELD = "ELAIS-S1"
catname = "../../dmu1/dmu1_ml_ELAIS-S1/data/master_catalogue_elais-s1_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [ ]:
z_bands = ["DECAM z", "VISTA Z"]
y_bands = ["DECAM y", "VISTA Y"]

all_bands = [z_bands, y_bands]

## 1. Magnitudes and magnitude erros

In [ ]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 2.c IRAC Aperture magnitudes

In [ ]:
irac_mag = 3.9000000001085695
bands = ["IRAC1", "IRAC2", "IRAC3", "IRAC4"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap = "merr_ap_{}".format(basecol)
    flagcol_ap = "flag_{}_ap".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == irac_mag)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)

## 2. Outliers

In [ ]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        
        master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
                      
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
                      
        master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))            
                      

## 3. Save table

In [ ]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)